In [0]:
############## Use Spark Streaming for this task ##############

# SQL connection works well 
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W

# SQL server connection Details ----------
server = 'sqldbplatingnum.database.windows.net'
database = 'cozydb'
username = 'platingnum'
password = 'D0wnl0ad!'
driver= '{ODBC Driver 13 for SQL Server}'
connectionProperties = {
  "user" : username,
  "password" : password
}

# File Processing   
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("/FileStore/tables/Building_Maintenance__1_-c86c7.csv")
df.registerTempTable("foo")

# Save data from dataframe to SQL server table cozy_transaction - can take hour  - Uncomment all 4 lines 
connection_string = 'Driver=' + driver + ';Server=' + server + ';Database=' + database + ';Trusted_Connection=Yes;UID=' + username + ';PWD=' + password + ';'
jdbc_url = "jdbc:sqlserver://"+ server +";database="+ database +";user="+ username +";password="+ password
df.write.jdbc(jdbc_url, 'foo', mode='overwrite', properties=connectionProperties)

In [0]:
# There is an issue with ADL (Azure Data lake) and Trying to fix it 
# ADL connection Details 
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", "1c41fd5a-7ad8-462c-9168-7f8ed7195905")
spark.conf.set("dfs.adls.oauth2.credential", "M/y7C/PCDT+CJRdcnz27hQbW7lO5BQA3H9WAIz34Vxflm15R9W6TyzHOzig3XN220mrErdDYictnETz2njlB6g==")
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/1c41fd5a-7ad8-462c-9168-7f8ed7195905/oauth2/token")

In [0]:
Test Data produced  at https://my.api.mockaroo.com/users_load.json?key=6af9c3e0

  ############################# Matching rules #############################
  A relation is NOT considered matching if only ONE of the following combo items is matched
  Require 2 or more of rules below 1,2,3,4,7,8,9 and one match for rules 5-6
    1: Firstname + IP
    2: Firstname + Username
    3: Firstname + DOB + City
    4: Firstname + Postcode
    7: DOB + Postcode
    8: Address1 + Postcode
    9: Firstname + Lastname + Address1 + City

  Is a full match if any 2 or more matched combinations, and/or a single match on
    5: Firstname + Mobile or
    6: Email

  If RelatedID <> 0 (meaning its a modification), log it and if there is no new   related user it will get a RelatedID = 0 
  If there is a new relation then it gets the 1st users its related to RelatedID
  If there of the existing related player, or if that related player has no 
  RelatedID then create a new RelatedID    for them both
  ############################# Matching rules #############################
  SQL Input table 
CREATE TABLE users_load(
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[userID] [int] NULL,
	[SkinID] [int] NULL,
	[Username] [nvarchar](15) NULL,
	[Email] [nvarchar](100) NULL,  -- check email in right email format 
	[Firstname] [nvarchar](50) NULL,  -- Check firstname is at least 2 char and don't have any number, special char, under scores etc and remove them
	[Lastname] [nvarchar](50) NULL, -- Check lastname is at least 2 char and don't have any number, special char, under scores etc and remove them
	[RegDate] [datetime2](7) NULL,  -- Date is in Date format 
	[RegIP] [nvarchar](50) NULL,
	[LastIP] [nvarchar](50) NULL,
	[DOB] [nvarchar](8) NULL,   -- DOB is in Date format 
	[Postcode] [nvarchar](15) NULL,   
	[MobilePhone] [nvarchar](50) NULL,    -  In format of 10 numbers  07123123456 
							    if country codes are there remove them +4407123123456  becomes  4407123123456 
							    +07123123456  becomes 07123123456 , (07123123456)   becomes 07123123456 
							0712-3123-456 becomes 07123123456 
	[landline] [nvarchar](50) NULL,        - In format of 11 numbers starting with 01X, 02X, 03X  use same cleansing as Mobile numbers  
	[Address1] [nvarchar](100) NULL,    -  Don't have any number, special char else  remove them
	[Address2] [nvarchar](100) NULL,    -  Don't have any number, special char else  remove them
	[City] [nvarchar](50) NULL,          -   Don't have any number, special char else  remove them
	[SelfExcludedUntil] [datetime2](0) NULL,  -- Format date in Std format 
	[Status] [int] NULL,                        -  Don't have any number, special char else  remove them 
 CONSTRAINT [PK_users_Load] PRIMARY KEY CLUSTERED 
(
	[ID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

Output Table 
CREATE TABLE UserSCV(
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[EntityID] [int] NOT NULL,
	[RelatedID] [int] NOT NULL,
	[Username] [nvarchar](50) NOT NULL,
	[Email] [nvarchar](100) NOT NULL,
	[OriginalEmail] [nvarchar](100) NOT NULL,
	[Firstname] [nvarchar](50) NOT NULL,
	[OriginaFirstname] [nvarchar](50) NOT NULL,
	[Lastname] [nvarchar](50) NOT NULL,
	[OriginaLastname] [nvarchar](50) NOT NULL,
	[RegDate] [datetime2](7) NULL,
	[OriginaLRegDate] [datetime2](7) NULL,
	[RegIP] [nvarchar](50) NOT NULL,
	[LastIP] [nvarchar](50) NOT NULL,
	[DOB] [datetime2](8) NOT NULL,
	[OriginaLDOB] [nvarchar](8) NOT NULL,
	[Postcode] [nvarchar](15) NOT NULL,
	[OriginaLPostcode] [nvarchar](15) NOT NULL,
	[MobilePhone] [int] NOT NULL,
	[OriginaLMobilePhone] [nvarchar](50) NOT NULL,
	[Address1] [nvarchar](250) NOT NULL,
	[OriginaLAddress1] [nvarchar](250) NOT NULL,
	[Address2] [nvarchar](250) NOT NULL,
	[OriginaLAddress2] [nvarchar](250) NOT NULL,
	[City] [nvarchar](100) NULL,
	[OriginaLCity] [nvarchar](100) NULL,
	[SelfExcludedUntil] [datetime2](0) NULL,
	[Status] [int] NOT NULL,
	[Firstname_Lastname_RegIP] [varbinary](900) NULL,                    - A hash string created using these column combinations 
	[Firstname_Lastname_LastIP] [varbinary](900) NULL,                   - A hash string created using these column combinations 
	[Firstname_Lastname_Username] [varbinary](900) NULL,                       -- A hash string created using these column combinations 
	[Firstname_DOB_City] [varbinary](900) NULL,                          - A hash string created using these column combinations 
	[Firstname_Postcode] [varbinary](900) NULL,                          - A hash string created using these column combinations 
	[Firstname_Mobilephone] [varbinary](900) NULL,                       - A hash string created using these column combinations 
	[Email_Hash] [varbinary](900) NULL,                                            -- A hash string created using these column combinations 
	[DOB_Postcode] [varbinary](900) NULL,                                           -- A hash string created using these column combinations 
	[Address1_Postcode] [varbinary](900) NULL,                       -- A hash string created using these column combinations 
	[Firstname_Lastname_Address1_City] [varbinary](900) NULL,                       -- A hash string created using these column combinations 
	[LoadDate] [date] NOT NULL, 
	[LastModifiedDate] [date] NOT NULL,
	[CompareStatus] [tinyint] NOT NULL,
	[State] [nvarchar](100) NULL,
 CONSTRAINT [PK_User] PRIMARY KEY CLUSTERED 
(
	[ID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO


In [0]:
from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, DoubleType, StructType, StructField
import requests
import json
import phonenumbers 
import re
import datetime 

In [0]:
def fixUserRow(c):
    # get the Mobile field
    number = c.MobilePhone

    # initialize variables 
    is_valid_number = "N"
    clean_number = None
    number_type = None
    valid_mail = None

    p = None
	#      [MobilePhone] [nvarchar](50) NULL,    -  In format of 10 numbers  07123123456 
	#      						    if country codes are there remove them +4407123123456  becomes  4407123123456 
	#      						    +07123123456  becomes 07123123456 , (07123123456)   becomes 07123123456 
	#      						0712-3123-456 becomes 07123123456 
	#      [landline] [nvarchar](50) NULL,        - In format of 11 numbers starting with 01X, 02X, 03X  use same cleansing as Mobile numbers  
    
    if number is not None:
        # Clean the Mobile Number first
        try:
            p = phonenumbers.parse(number, None)

            if phonenumbers.is_valid_number(p):
                is_valid_number = "Y"
            elif phonenumbers.truncate_too_long_number(p):
                is_valid_number = "Y"
            else:
                is_valid_number = "N"

            clean_number = "%s%s" % (p.country_code, p.national_number)
            
        except:
            p = None

    # clean up PhoneNumber
    phone_no = c.Landline
    if phone_no is not None:
      phone_no = phone_no.replace('-', '')
      if (len(phone_no) != 10):
        phone_no = None
    
    # validate Email 
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", c.email):
      valid_mail = c.email
    
    return Row( 
		id = c.id, 
        Userid = c.Userid, 			
        SkinID = c.SkinID, 			
        first_name = c.first_name, 		
        last_name = c.last_name,	
        email = valid_mail,			
        gender = c.gender,			
        ip_address = c.ip_adddress, 		
		LastIP = c.LastIP,			
		DOB = c.DOB,			
		Postcode = c.Postcode,		
		MobilePhone = clean_number, 	
		Landline = phone_no, 		
		Address1 = c.Address1,		
        City = c.City, 			
		County = c.County,			
		Country = c.Country, 		
      ``SelfExcludedUnttil = c.SelfExcludedUnttil
		Status = c.Status			
    )

  

In [0]:
#df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("https://my.api.mockaroo.com/users_load.json?key=6af9c3e0") 

csv_url = "https://my.api.mockaroo.com/users_load.json?key=6af9c3e0"

import pandas as pd

df_user = spark.createDataFrame(pd.read_csv(csv_url))

# cleanse the data before inserting into db



In [0]:
df_user.show(5)

+---+------+----------+---------+----------+---------+--------------------+------+---------------+----------+--------------+---------------+----------+--------+-------------+------------+--------------------+--------+------+-------+-----------------+------+
 id|Userid| SkinID| username|first_name|last_name| email|gender| ip_address| RegDate| RegIP| LastIP| DOB|Postcode| MobilePhone| Landline| Address1| City|County|Country|SelfExcludedUntil|Status|
+---+------+----------+---------+----------+---------+--------------------+------+---------------+----------+--------------+---------------+----------+--------+-------------+------------+--------------------+--------+------+-------+-----------------+------+
 1| 1| Pink|skelwick0| Saidee| Kelwick| skelwick0@noaa.gov|Female| 157.17.140.68|04/30/2017| 80.180.4.38| 87.162.178.115|12/17/1973| 12232|(518) 6508290|212-763-1805|9817 Division Avenue| Albany| NY| US| 7/9/2017| true|
 2| 2| Turquoise|lsunners1| Laurens| Sunners| lsunners1@house.gov| Male| 188.141.3.232|02/24/2017| 144.59.200.98| 11.119.63.18|06/15/1967| 10459|(917) 4832437|585-831-0600| 28163 Carioca Lane| Bronx| NY| US| 6/11/2017| false|
 3| 3|Aquamarine|astaning2| Alano| Staning|astaning2@usatoda...| Male|141.131.227.198|07/03/2017| 195.218.29.26| 142.254.171.8|08/26/1983| 12262|(518) 4439387|718-822-0364| 9359 Kipling Point| Albany| NY| US| 5/10/2017| true|
 4| 4| Orange| pdicey3|Petronilla| Dicey|pdicey3@pinterest...|Female| 14.176.215.114|08/16/2017| 158.9.191.193| 55.211.110.79|03/09/1996| 12255|(518) 8296357|347-770-1410| 7458 Namekagon Road| Albany| NY| US| 8/13/2017| false|
 5| 5| Yellow| cgullis4| Clayson| Gullis|cgullis4@angelfir...| Male| 89.161.156.105|10/18/2017|125.76.110.248|154.126.139.103|11/26/1979| 11205|(718) 2626129|518-811-2547| 57637 Manley Place|Brooklyn| NY| US| 1/15/2018| true|
+---+------+----------+---------+----------+---------+--------------------+------+---------------+----------+--------------+---------------+----------+--------+-------------+------------+--------------------+--------+------+-------+-----------------+------+
only showing top 5 rows

In [0]:
df_user.write.jdbc(jdbc_url, 'users_load', mode='overwrite', properties=connectionProperties)

In [0]:
df_user_updated = df_user.rdd.map(lambda c: fixUserRow(c))
